# Data Workflow

The main workflow for any data analysis project involves the following:

* R Set-up (loading relevant packages and data)
* Data Wrangling (cleaning and preparation)
* Visualisations (tables and graphs)
* Analysis (modelling or other technical analysis)
* Assumption checking

```{note}
Advanced statistical analysis is beyond the scope of this workshop.
```

<a id="scenario"></a>
## Scenario

We are a part of a research team conducting a study that aims to understand if there is an association between the average number of hours an individual sleeps each night `[sleep_hrs_night]` and the development of hypertension
`[hypertension]` in adults aged 40 years `[age]` or older. The research team believes that the following factors could obscure such an association, and as such should be adjusted for:

* Average daily alcohol consumption `[alcohol_day]`
* Regular use of marijuana `[regular_marij]`
* Is participant currently a smoker `[smoke_now]`
* Is participant regularly physically active `[phys_active]`
* BMI Group (underweight, healthy, overweight, obese) `[bmi]`

To answer this research question, we will use the US National Health and Nutrition Examination Survey (NHANES) provided within the data file `sleep.csv.`


```{note}
The statistical methods for survey data are beyond the scope of this R workshop. As such we will treat this data as if it was collected from randomly selected individuals.
```

# R Set-up

There are a few things you need to do every time you open R:

* Install/Load packages
* Read in data (import data into R)


## Install/Load Packages



We **install** packages in R once. We **load** packages we are going to use everytime we open/start R. We recommended that you write the installation/load code at the top of your code script. This makes it easy to see what packages you used at a glance.

```{tip}
The symbol # is used to indicate a comment in R. This is a line which R will skip over and not try to run as code. The best codes are well commented.
```

In [ ]:
# Install your packages (first use only)
install.packages("tidyverse")
install.packages("janitor")
install.packages("gtsummary")
install.packages("broom.helpers")

# Load your packages (everytime you restart R)
library(tidyverse)
library(janitor)
library(gtsummary)
library(broom.helpers)

## Load in your Data



There are a variety of different ways to read data into R, since there are a variety of different ways to save and store data. One of the more common ways to save data is as a CSV (comma delimited file).

```{tip}
You will need to know which directory (folder) you're working in and point correctly to the file sleep.csv. See [Path Setting](pathsetting.md) for additional information.
```

In [ ]:
# Read in your data and assign this to a dataframe called sleep
# A dataframe is the object R uses to store data sets
sleep <- read_csv("sleep.csv")

```{note}
Notice the message R prints out. It indicates the delimiter (,), the type of each column, and the row and column count.
```

```{tip}
R automatically saves the first row from the .csv file as column names. Ensure only the first row and first row of your data file describe the variables.
```

# Data Wrangling

## Viewing your data

It is generally **good practice** to look at your data to scan for things that might need fixing, for example:

* text in numerical cells (e.g. “six” vs 6)
* odd characters (e.g. >, <, /, etc.)
* inconsistent Date formats (e.g 01/16/2024, 16-1-24, Jan 16 2024, etc.)
* coded variables (e.g. 99 = Missing)
* typos (e.g. ys, yse)
* inconsistent capitalisation or abbreviations (e.g. Yes, Y, yes)
* misaligned/counterintuertive variable types

```{note}
Variables types include:
* numeric: doubles (dbl) - decimals, or integers (int) - whole numbers
* characters (chr) - text
* logicals (lgl) - “TRUE” or “FALSE”
* factors (fct) - categorical data stored as integers (e.g. Strongly Agree = 5)
```

```{Caution}
It is **bad practice** to look at your data to determine patterns or to determine what research questions you should ask and test.
```

There are a couple of different ways you can look at your data:


In [ ]:
# glimpse provides information on each column, it's name, type and the first few entries:
glimpse(sleep)

In [ ]:
# head (tail) shows the first (last) n rows:
# head(sleep, n=2)
# tail(sleep, n=3)

In [ ]:
# View will open the whole dataframe
# We will comment this out here to save space
# View(sleep) # alternatively in RStudio click on sleep in the environment panel

## Data Cleaning and Preparation

### Cleaning column names

A good first step is to clean column names, in case there are any odd symbols or mixed cases. The quickest way to do this is with the clean_names function from the `janitor` package.

In [ ]:
# fix column names reassigning this to the dataframe sleep
sleep <- clean_names(sleep)

# glimpse the dataframe to see the changes that have been made
glimpse(sleep)

### Trimming the data

It is often practical to trim your data set down to only the
necessary columns (variables). We can easily do this using the `select` function. We will keep the variables mentioned in the [Scenario](#scenario). We will also keep the gender variable `[gender]` too as there may be interesting sex-stratified differences in the data. It is also important to keep the identifier `[id]` for each data entry.

```{tip}
It is best practise to preserve the original dataframe so when we trim we assign this to a new dataframe.
```

In [ ]:
# trim sleep to select columns only and assign this to a new dataframe sleep_trim
sleep_trim <- sleep %>%
  select(id,
    gender,
    age,
    hypertension,
    sleep_hrs_night,
    phys_active,
    smoke_now,
    alcohol_day,
    regular_marij,
    bmi)

# glimpse the new dataframe
glimpse(sleep_trim)

### Filtering the data

We also want to filter the data down to our cohort of interest. In this case, we are interested in adults 40
years or older. We can use the `filter` function to remove participants who are younger than 40 from our data set, or rather only keep participants who are onlder than 40.

```{warning}
It would be a good idea check a numerical summary of the age variable to make sure there are no extreme or impossible values before we filter
```

In [ ]:
# generate a summary of the age variable belonging to the sleep_trim dataframe
summary(sleep_trim$age)

This looks reasonbale and so we can proceed to filter.

In [ ]:
# filter sleep_trim to select entries we want to keep assigned to a new dataframe
sleep_40 <- sleep_trim %>%
filter(age >= 40) #only those rows with age>=40

# add a check that the filter worked
cat("The minimum age of our data set is:", min(sleep_40$age), "\n")

```{tip}
Print statements are a valuable tool when coding (in R or otherwise). Here we make use of the `cat` function so it is clear what the R output is telling us.
```

### Variable checking

We will go through each of our variables in turn and check for required fixes.

```{tip}
As a rule of thumb:
* if we are checking a continuous variable we can use `summary` which tells you about the distribution of values for that variable.
* if we are checking a discrete or catergorical variable we can use the `table` function which tabulates counts of each response.
```

We will first consider our exposure variable, in this case, the number of hours a participant sleeps each night `[sleep_hrs_night]`. This is a continuous variable so we will check to see if the summary values look reasonable.

In [ ]:
# generate a summary of sleep_hrs_night variable
summary(sleep_40$sleep_hrs_night)

```{tip}
Although the values look reasonable, 4 hours is not a lot of sleep. In practice, it might be worth checking with the data provider to make sure that there is not a typo somewhere.
```

Next we will check our outcome variable, in this case whether a participant has hypertension or not `[hypertension]`. This is a catergorical variable so we can tabulate the responses.

In [ ]:
# create a table of values for hypertension
table(sleep_40$hypertension)

This is good there are two possible responses as we would expect.

Now let's fix the `[alcohol_day]` variable, which we saw earlier was classed a character variable. We will tabulate the variable to see what is going on.

In [ ]:
# create a table of values for alcohol_day
table(sleep_40$alcohol_day)

We can see each response (e.g 1 drink a day) is listed with the respective count (e.g. 1007 have 1 drink a day). We notice that some values were written as text. We need to replace the values written as text and reclassify the variable as numeric.

```{tip}
Since best practice is to maintain the original data we will create a new variable using the `mutate` function.
```

In [ ]:
# create a numeric alcohol drink count variable called alcohol_day_num
sleep_40 <- sleep_40 %>%
mutate(alcohol_day_num = case_when(alcohol_day == "one" ~ 1, # if alcohol_day is one replace with 1
                                   alcohol_day == "One" ~ 1,
                                   alcohol_day == "three" ~ 3,
                                   alcohol_day == "Three" ~ 3,
                                   alcohol_day == "thre" ~ 3,
                                   alcohol_day == "Six" ~ 6,
                                   TRUE ~ as.numeric(alcohol_day)))

```{warning}
R warns us that since we classified `[alcohol_day_num]` as a numeric variable any remaining text values will be replaced with missing values (NA) so we should check to see if we missed anything by tabulating this new variable.
```

A slightly tidier version of this code makes use of the `%in%` and `c()` functions

In [ ]:
# create a numeric alcohol drink count variable called alcohol_day_num
sleep_40 <- sleep_40 %>%
mutate(alcohol_day_num = case_when(alcohol_day %in% c("one", "One") ~ 1, #replace one or One with 1
                                   alcohol_day %in% c("three", "Three", "thre") ~ 3,
                                   alcohol_day == "six" ~ 6,
                                   TRUE ~ as.numeric(alcohol_day)))

In [ ]:
# create a table of values for alcohol_day_num
table(sleep_40$alcohol_day_num)

# check the class of alcohol_day_num
cat("alcohol_day_num is of type:", class(sleep_40$alcohol_day_num), "\n")

```{tip}
It would be a good idea to check the counts of `[alcohol_day_num]` correctly align with those from `[alcohol_day]`
```

Now let us check the remaining variables

In [ ]:
# check marijuana indicator variable
table(sleep_40$regular_marij)

In [ ]:
# check smoker indicator variable
table(sleep_40$smoke_now)

In [ ]:
# check physical activity indicator variable
table(sleep_40$phys_active)

All three of these variables are binary variables with only yes and no responses (with no typos!).

```{note}
Such data could equally be stored as a logical variable where for example, `smoke_now==True` was equivalent to `smoke_now==Yes`.
```

Moving onto the `[bmi]` data. As this data is likely to take continuous rather than discrete values we would use the `summary` function rather than tabulating all values.

In [ ]:
# generate a summary of bmi variable
summary(sleep_40$bmi)

This seems reasonable, but the [Scenario](#scenario) listed BMI groups (e.g. underweight, healthy, overweight, obese) rather than the BMI. Let us create a new bmi variable to aggregate participants into their BMI group.

In [ ]:
# create a discrete BMI variable called bmi_group
sleep_40 <- sleep_40 %>%
mutate(bmi_group = case_when(bmi < 18.5 ~ "Underweight",
                             bmi >= 18.5 & bmi < 25 ~ "Healthy",
                             bmi >= 25 & bmi < 30 ~ "Overweight",
                             bmi >= 30 ~ "Obese",
                             TRUE ~ NA)) # any values that are not captured by the conditions are set as missing

Now that we have a discrete variable `[bmi_group]` we can check this using the `table` function and check we don't have any missing values.

In [ ]:
# create a table of values for bmi_group
table(sleep_40$bmi_group)

We can also check that we got our groupings correct. The following lines of code show that our minimum and maximum values all line up with the correct BMI groupings.



In [ ]:
sleep_40 %>%
group_by(bmi_group) %>%
summarise(min_bmi = min(bmi),
          max_bmi = max(bmi)) %>%
            arrange(min_bmi) # arrange in ascending min_bmi order (default alphabetical)

# Visualisations

Now that the data is cleaned and ready to go we can start creating our data summaries and visuals.

## Tables

Let’s create a table for the research project. A table should include all the variables of interest in the study and thus, describes the sample.
We will use the `select` function to pick the variables to include in the table and the `tbl_summary` function from the `gtsummary` package to create the table.


```{tip}
It is not uncommon to miss something in the initial data cleaning, so carefully examine all the tables and graphs you make. If you do notice some data cleaning step that you missed, it’s a good idea to add the missed step to the original data cleaningcode, if possible, to keep your code organised.
```

In [ ]:
# create a table and assign it to the object table_1
table_1 <- sleep_40 %>%
  # select the variables to include
  select(gender,
         age,
         phys_active,
         smoke_now,
         alcohol_day_num,
         regular_marij,
         bmi_group,
         hypertension,
         sleep_hrs_night) %>%
           display_html(tbl_summary(missing_text = "(Missing)",
                # define clear labels for each variable
                label = list(gender ~ "Gender",
                age ~ "Age",
                phys_active ~ "Regularly Physically Activity",
                smoke_now ~ "Current Smoker",
                alcohol_day_num ~ "Alcohol Drinks per Day",
                regular_marij ~ "Regular Marijuana User",
                bmi_group ~ "BMI",
                hypertension ~ "Hypertension",
                sleep_hrs_night ~ "Sleep (hrs/night)")))

# display the table
table_1

## Graphs

We will create a simple graph of the average number of hours a participant slept per night grouped by the participants hypertension status. We will create a box plot, where separate boxes are used for participants with and without hypertension.
Let’s start with a basic box plot of hours of sleep per night and then build out graph up bit by bit from there.
We will use the `ggplot` function and `geom_boxplot`.

In [ ]:
# a simple boxplot of all the data
sleep_40 %>%
  ggplot(aes(x = hypertension, y = sleep_hrs_night)) +
  geom_boxplot()

This is a great start but we could make the graph more aesthetically pleasing. We could:

* make the axis labels clear
* change the font size
* add colour based on hypertension status
* change the background by updating the plot theme

In [ ]:
sleep_40 %>%
  ggplot(aes(x = hypertension, y = sleep_hrs_night, group = hypertension,
             colour = hypertension, fill = hypertension)) + # add colour
  geom_boxplot() +
  scale_colour_manual(values = c("#7c3b37", "#004c4e")) + # add defined outlines to boxes
  labs(x = "Hypertension Status", y = "Hours of Sleep/Night") + # fix labels
  guides(fill = "none", colour = "none") + # remove legend
  theme_bw() + # change background
  theme(text = element_text(size = 20)) # increase the font size